# Import packages et fichiers sources

In [5]:
#import des bilbiothèques nécessaires
import pandas as pd
import os
from itertools import permutations
import requests
import json
import polyline
import geopandas as gpd
import folium
import numpy as np
import time  
from shapely.geometry import LineString, Point
from shapely.wkt import loads as wkt_loads

# Afficher les versions pour vérifier que tout est bien installé
print("requests:", requests.__version__)
print("pandas:", pd.__version__)
print("geopandas:", gpd.__version__)
print("folium:", folium.__version__)
print("numpy:", np.__version__)

requests: 2.28.1
pandas: 2.2.3
geopandas: 1.0.1
folium: 0.19.5
numpy: 1.26.4


In [6]:
# Charger la liste des centroides des quartiers depuis un CSV (colonnes id-nom-latitude-longitude)
#df_lieux_csv = pd.read_csv("coords_quartiers_test.csv")
df_lieux_csv = pd.read_csv("input/coords_quartiers.csv")

#Charge la matrice de demande OD entre tous les quartiers (Attention à la bonne correspondance des noms)
file_path = "input/9000_2410-tab-gpe-TIM-2030PPS_OD_85quartiers_6corridors_by_Script.xlsx"  
df_matrice_OD = pd.read_excel(file_path, sheet_name="NbDépVP", index_col=0)

In [7]:
# Créer toutes les paires possibles sans les paires origine = destination
paires_lieux = list(permutations(df_lieux_csv.itertuples(index=False), 2))  # (85+6)*(85+6) - (85+6) = 8190 paires

# Clef d'appel API Google Maps (directions)

In [8]:
# doc ici : https://developers.google.com/maps/documentation/directions/overview?hl=fr
Clef = 'YOUR_API_KEY'

# Fonctions cache - sécurité

#### Les fonctions ci-dessous permettent de stocker l'information si une paire OD a deja été investiguée dans un cache. Si c'est le cas, l'API ne fera pas d'appel et cela permet d'éviter des appels inutiles

In [9]:
#fonction pour rajouter un itinéraire au cache
#def add_to_cache(origin, destination, route_info, cache, i, cache_file="cache_routes_test.json"):
def add_to_cache(origin, destination, route_info, cache, i, cache_file="cache_routes.json"):
    cache_key = f"{origin[0]}_{destination[0]}"
    cache[cache_key] = route_info  # Ajouter les données au cache
    save_cache(cache, cache_file, i)  # Sauvegarder le cache immédiatement
    
# Charger le cache à partir du fichier JSON (s'il existe)
#def load_cache(cache_file="cache_routes_test.json"):
def load_cache(cache_file="cache_routes.json"):
    try:
        with open(cache_file, "r") as f:
            content = f.read().strip()  # Lire et supprimer les espaces inutiles
            if not content:  # Si le fichier est vide
                return {}  # Retourner un dictionnaire vide
            cache = json.loads(content)
            for key, route in cache.items():
                route["start_location"] = Point(route["start_location"])
                route["end_location"] = Point(route["end_location"])
                route["geometry"] = wkt_loads(route["geometry"])
            return cache  # Charger les données du cache
    except FileNotFoundError:
        with open(cache_file, "w") as f:
            json.dump({}, f)  # Créer un fichier vide si pas existant
        return {}  # Si le fichier n'existe pas encore, retourner un cache vide

# Sauvegarder le cache dans le fichier JSON
def save_cache(cache, cache_file, i):
    attempts = 5  # Nombre de tentatives avant d'abandonner
    interval = 50 #Enregistrer dans le fichier tous les 50 itinéraires (soulage l'ouverture du cache)
    max_itineraire = 8190 #A modifier selon le nombre max d'itinéraires prévus afin d'intégrer le reliquat d'itinéaire (<50) dans le cache
    for j in range(attempts):
        if i % interval == 0 or i == max_itineraire :
            try:
                with open(cache_file, "w") as f:
                    json.dump(cache, f)
                    print("CACHED")
                return  
            except PermissionError:
                print(f"Erreur de permission, tentative {j+1}/{attempts}...")
                time.sleep(1)  # Attendre 1s avant de réessayer d'écrire dans le fichier json
            print("Impossible d'écrire dans le fichier json après plusieurs tentatives.")


# Vérifier si l'itinéraire entre les deux lieux est dans le cache
def is_cached(origin, destination, cache):
    # Créer la clé unique pour la paire de lieux (origin, destination)
    cache_key = f"{origin[0]}_{destination[0]}"
    return cache_key in cache  # Si la clé existe, l'itinéraire est dans le cache

# Récupérer un itinéraire depuis le cache
def get_from_cache(origin, destination, cache):
    cache_key = f"{origin[0]}_{destination[0]}"
    return cache.get(cache_key)  # Retourner les données du cache pour cette clé


# Fonction d'appel API Google Maps (directions)

In [10]:
# Fonction pour récupérer l'itinéraire
def get_route(origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{origin[2]},{origin[3]}",
        "destination": f"{destination[2]},{destination[3]}",
        "mode": "driving",
        "key": Clef
    }
    response = requests.get(url, params=params).json()
    
    # Vérifier s'il y a un itinéraire
    if response["status"] == "OK" : # si succès de la requête
        route = response["routes"][0]
        leg = route["legs"][0]

        # Extraire les données importantes
        distance = leg["distance"]["value"]
        duration = leg["duration"]["value"]
        start_address = leg["start_address"]
        end_address = leg["end_address"]
        encoded_polyline = route["overview_polyline"]["points"]
        decoded_polyline = polyline.decode(encoded_polyline)

        # Créer la géométrie LineString
        line_geom = LineString(decoded_polyline)
        
        return {
            "start_id": origin[0],
            "start_name": origin[1],
            "start_adresse": start_address,
            "start_location": [origin[2], origin[3]],
            "end_id": destination[0],
            "end_name": destination[1],
            "end_adresse": end_address,
            "end_location": [destination[2], destination[3]],
            "distance_m": distance,
            "duration_s": duration,
            "geometry": line_geom.wkt
        }
    else:
        return "Pas d'itinéraire trouvé pour";origin[1];destination[1]  # Retourne None si pas d'itinéraire

In [11]:
# Fonction principale pour récupérer l'itinéraire avec cache ou l'API (via fonction get_route)
def get_route_with_cache(origin, destination, cache, i):
    
    if is_cached(origin, destination, cache):
        print(f"Cache hit: {origin} -> {destination}")
        return get_from_cache(origin, destination, cache)  # Récupérer depuis le cache

    # Sinon, appeler l'API
    print(f"API request: {origin} -> {destination}")
    route_info = get_route(origin, destination)
    
    if route_info:
        add_to_cache(origin, destination, route_info, cache, i)  # Ajouter au cache
    return route_info

In [12]:
#Les coordonnées doivent être en général toujours au format longitude/latitude mais l'API maps donne latitude/longitude donc une fonction d'inversion permet de remttre les choses en ordre
def invert_coordinates(geom):
    if isinstance(geom, LineString):
        return LineString([(y, x) for x, y in geom.coords])  # Inversion (lat, lon) -> (lon, lat)
    if isinstance(geom, Point):
        return Point([(y, x) for x, y in geom.coords])  # Inversion (lat, lon) -> (lon, lat)
    return geom

In [13]:
def get_demand(origin, destination, matrix):
    try:
        return matrix.loc[origin, destination]
    except KeyError:
        return 9999  # Si la paire OD n'existe pas, attribuer 0 --> permettra de trouver les erreurs d'association au besoin

# Boucle sur toutes les paires OD pour récupérer les polylignes

!!! NE PAS RUN SANS ETRE SUR DE LA PERTINENCE : BOUCLE AVEC APPEL API PAYANT !!!

In [14]:
# Charger le cache existant (si disponible)
cache = load_cache()

routes_data = []  # Stocke les itinéraires

remove_chars = [')', '(', "'"]

i = 0 #(compteur des itinéraires appelés)

for origin_str, destination_str in paires_lieux:
    
    # Découper la chaîne pour obtenir les différentes valeurs (passage du dictionnaire tuple en chaine de caractère propre)
    origin_str_converted = f"{origin_str}"
    destination_str_converted = f"{destination_str}"
    for char in remove_chars: #nettoyage de la chaien de caractère
        origin_str_converted = origin_str_converted.replace(char, '')
        destination_str_converted = destination_str_converted.replace(char, '')

    origin_parts = origin_str_converted.split(';')  # crèe 4 champs distinct
    destination_parts = destination_str_converted.split(';')  # idem
    
    
    # Extraire les informations nécessaires
    origin_id = origin_parts[0]
    origin_name = origin_parts[1]
    origin_latitude = float(origin_parts[2])  # Convertir en float pour les coordonnées
    origin_longitude = float(origin_parts[3])
    
    destination_id = destination_parts[0]
    destination_name = destination_parts[1]
    destination_latitude = float(destination_parts[2])
    destination_longitude = float(destination_parts[3])

    # Afficher pour vérifier
#     print(f"Origin: {origin_name} ({origin_latitude}, {origin_longitude})")
#     print(f"Destination: {destination_name} ({destination_latitude}, {destination_longitude})")
    
    i=i+1
    route_info = get_route_with_cache(origin_parts, destination_parts, cache, i) #APPEL API ICI
    
    if route_info:
        routes_data.append(route_info)  # Ajouter les résultats valides
    
    time.sleep(0.5)  # Pause pour éviter de surcharger l'API Google et de bien enregistrer le cache, 2e sécurité en + du cache

for route in routes_data:
    if isinstance(route["geometry"], str):  # Vérifie si c'est une chaîne WKT
        route["geometry"] = wkt_loads(route["geometry"])
    elif not isinstance(route["geometry"], LineString):  # Vérifie qu'on a bien une LineString
        raise TypeError(f"Valeur inattendue pour 'geometry': {route['geometry']}")
        
# Créer le GeoDataFrame final avec correction de la forme des coordonnées (lat/long > long/lat) et association de la demande à chaque paire OD
gdf_routes = gpd.GeoDataFrame(routes_data, geometry="geometry", crs="EPSG:4326")
gdf_routes["geometry"] = gdf_routes["geometry"].apply(invert_coordinates)
gdf_routes["start_location"] = gdf_routes["start_location"].apply(invert_coordinates)
gdf_routes["end_location"] = gdf_routes["end_location"].apply(invert_coordinates)

gdf_routes["demande"] = gdf_routes.apply(lambda row: get_demand(row["start_name"], row["end_name"], df_matrice_OD),axis=1)

print(gdf_routes.head())
print("Fin de création des polylignes")


In [15]:
# Sauvegarde en GeoJSON (format léger)
gdf_routes.to_file("output/itineraire_global_TIM.geojson", driver="GeoJSON")
#gdf_routes.to_file("output/itineraire_test_TIM.geojson", driver="GeoJSON")

# Sauvegarde en Shapefile (compatible QGIS/ArcGIS)
gdf_routes.to_file("output/itineraire_global_TIM.shp")
#gdf_routes.to_file("output/itineraire_test_TIM.shp")
print("Fichiers créés")

C:\Users\gpe\AppData\Local\Temp\ipykernel_34128\2027635788.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_routes.to_file("output/itineraire_global_TIM.shp")
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'start_adresse' to 'start_adre'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'start_location' to 'start_loca'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'end_adresse' to 'end_adress'
  ogr_write(
C:\Users\gpe\AppData\Roaming\Python\Python39\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'end_location' to 'end_locati'
  ogr_write(


Fichiers créés


# Affichage interactif des résultats

In [20]:
# Créer une carte centrée sur le premier lieu
m = folium.Map(location=[gdf_routes.geometry[0].centroid.y, gdf_routes.geometry[0].centroid.x], zoom_start=12)

# Ajouter les itinéraires à la carte
for _, row in gdf_routes.iterrows():
    coords = [(point[1], point[0]) for point in row.geometry.coords]  # (lat, lon), folium utilise latitude en 1er
    folium.PolyLine(coords, color="blue", weight=2, opacity=0.4).add_to(m)

# Afficher la carte
#m #A décommenter pour afficher mais peut faire ramer si beaucoup de lignes à afficher

## Conversion en script .py 

In [17]:
#!jupyter nbconvert --to script extraction_polyligne_TIM.ipynb

In [18]:
len(cache)

8190